In [1]:
import dataclasses
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Dict, Optional

import numpy as np

In [2]:
from transformers import (
    AutoConfig,
    AutoTokenizer,
    T5Tokenizer,
    EvalPrediction,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    T5ForConditionalGeneration,
    set_seed )

In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [4]:
from utils_glue_T5_classification import (
    T5GlueDataset, 
    T5_glue_compute_metrics, 
    T5_glue_output_modes, 
    T5_glue_tasks_num_labels,
    processors,
    Split
)

In [5]:
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
    )
        
@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    task_name: str = field(metadata={"help": "The name of the task to train on: " + ", ".join(processors.keys())})
    data_dir: str = field(metadata={"help": "Should contain the data files for the task."})
    max_seq_length: int = field(
        default=128,
        metadata={
            "help": "The maximum total input sequence length after tokenization. Sequences longer "
            "than this will be truncated, sequences shorter will be padded."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    
    def __post_init__(self):
        self.task_name = self.task_name.lower()   

parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))
logger = logging.getLogger(__name__)

MODEL_NAME = "t5-small"
DATESTAMP = "20200814"
SUPER_GLUE_DIR = "/home/keyur/medhas/glue_data/"
TASK_NAME = "CoLA"
PER_DEVICE_BATCH_SIZE = 48
EXPERIMENT_DIR="/mnt/data/medhas/glue_experiments/%s/%s"%(MODEL_NAME, DATESTAMP)

custom_sysargv = [
"--model_name_or_path=%s"%MODEL_NAME,
"--task_name=%s"%TASK_NAME,
"--do_train",
"--do_eval",
"--data_dir=%s"%os.path.join(SUPER_GLUE_DIR, TASK_NAME),
"--max_seq_length=256",
"--per_device_train_batch_size=%s"%PER_DEVICE_BATCH_SIZE,
"--per_device_eval_batch_size=%s"%PER_DEVICE_BATCH_SIZE,
"--learning_rate=2e-5",
"--num_train_epochs=100",
"--output_dir=%s"%os.path.join(EXPERIMENT_DIR, TASK_NAME),
"--logging_dir=%s/logs"%os.path.join(EXPERIMENT_DIR, TASK_NAME),
"--logging_steps=90",
"--evaluate_during_training",
"--eval_step=90",
"--save_total_limit=2",
"--save_steps=1000",
"--gradient_accumulation_steps=1",
"--overwrite_output_dir"
]

model_args, data_args, training_args = parser.parse_args_into_dataclasses(args=custom_sysargv)

# Setup logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.WARN if training_args.local_rank in [-1, 0] else logging.WARN,
)
logger.warning(
    "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
    training_args.local_rank,
    training_args.device,
    training_args.n_gpu,
    bool(training_args.local_rank != -1),
    training_args.fp16,
)
logger.info("Training/evaluation parameters %s", training_args)

set_seed(training_args.seed)
training_args.seed
num_labels = T5_glue_tasks_num_labels[data_args.task_name]
output_mode = T5_glue_output_modes[data_args.task_name]
print ("Task:", data_args.task_name, "Labels:", num_labels, ', Output', output_mode)


08/25/2020 16:02:58 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 2, distributed training: False, 16-bits training: False


Task: cola Labels: 2 , Output classification


In [6]:
T5_glue_tasks_num_labels

{'sst-2': 2, 'cola': 2}

In [7]:
config = AutoConfig.from_pretrained(
        model_args.config_name if model_args.config_name else     model_args.model_name_or_path,
        num_labels=num_labels,
        finetuning_task=data_args.task_name,
        cache_dir=model_args.cache_dir,
    )

In [8]:
tokenizer = AutoTokenizer.from_pretrained(
        model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
    )

In [9]:
model = T5ForConditionalGeneration.from_pretrained(
        model_args.model_name_or_path,
        from_tf=bool(".ckpt" in model_args.model_name_or_path),
        config=config,
        cache_dir=model_args.cache_dir,
)

In [10]:
train_dataset = T5GlueDataset(data_args.data_dir, tokenizer=tokenizer, 
                    task=data_args.task_name, max_seq_length=data_args.max_seq_length, 
                    overwrite_cache=data_args.overwrite_cache, mode=Split.train,) if training_args.do_train else None

eval_dataset = T5GlueDataset(data_args.data_dir, tokenizer=tokenizer, 
                    task=data_args.task_name, max_seq_length=data_args.max_seq_length, 
                    overwrite_cache=data_args.overwrite_cache, mode=Split.dev,) if training_args.do_eval else None

test_dataset = T5GlueDataset(data_args.data_dir, tokenizer=tokenizer, 
                    task=data_args.task_name, max_seq_length=data_args.max_seq_length, 
                    overwrite_cache=data_args.overwrite_cache, mode=Split.test,) if training_args.do_predict else None

In [11]:
def compute_metrics(p: EvalPrediction) -> Dict:
    if output_mode == "classification":
        preds = np.argmax(p.predictions, axis=2)
    elif output_mode == "regression":
        preds = np.squeeze(p.predictions)
    return T5_glue_compute_metrics(data_args.task_name, preds, p.label_ids)

In [12]:
# Initialize our Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [13]:
if training_args.do_train:
        trainer.train(
            model_path=model_args.model_name_or_path if os.path.isdir(model_args.model_name_or_path) else None
        )
        trainer.save_model()
        # For convenience, we also re-save the tokenizer to the same directory,
        # so that you can share your model easily on huggingface.co/models =)
        if trainer.is_world_master():
            tokenizer.save_pretrained(training_args.output_dir)

/home/keyur/anaconda3/envs/medhas02/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'loss': 3.491254238287608, 'learning_rate': 1.98e-05, 'epoch': 1.0, 'step': 90}



{'eval_loss': 0.8428999348120256, 'eval_mcc': 0.0, 'epoch': 1.0, 'step': 90}



/home/keyur/anaconda3/envs/medhas02/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


{'loss': 0.8509377612007989, 'learning_rate': 1.9600000000000002e-05, 'epoch': 2.0, 'step': 180}


...


{'eval_loss': 0.6144245564937592, 'eval_mcc': 0.384964967431684, 'epoch': 98.0, 'step': 8820}



{'loss': 0.37660027113225725, 'learning_rate': 2.0000000000000002e-07, 'epoch': 99.0, 'step': 8910}



{'eval_loss': 0.6141540733250704, 'eval_mcc': 0.38790044127674095, 'epoch': 99.0, 'step': 8910}



{'loss': 0.38623579376273687, 'learning_rate': 0.0, 'epoch': 100.0, 'step': 9000}



{'eval_loss': 0.6140546663240953, 'eval_mcc': 0.38790044127674095, 'epoch': 100.0, 'step': 9000}




In [27]:
eval_dataloader = trainer.get_eval_dataloader(eval_dataset)

In [15]:
model = trainer.model

In [16]:
inputs = eval_dataloader.__iter__().next()

In [17]:
loss, logits, labels = trainer.prediction_step(model, inputs, False)

In [18]:
logits.shape

torch.Size([96, 1, 32128])

In [19]:
preds = logits
label_ids = labels
preds = preds.cpu().numpy()
label_ids = label_ids.cpu().numpy()
p = EvalPrediction(predictions=preds, label_ids=label_ids)
preds = np.argmax(p.predictions, axis=2)

In [22]:
matthews_corrcoef(label_ids, preds)

0.024539928613451497

In [23]:
model = trainer.model

In [32]:
import torch
from typing import Any, Callable, Dict, List, Optional, Tuple, Union

preds: torch.Tensor = None
label_ids: torch.Tensor = None
eval_losses: List[float] = []

In [26]:
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Dro

In [33]:
from tqdm.auto import tqdm, trange
import torch
description = "Evaluation"
prediction_loss_only = False
for inputs in tqdm(eval_dataloader, desc=description):
    loss, logits, labels = trainer.prediction_step(model, inputs, prediction_loss_only)
    if loss is not None:
        eval_losses.append(loss)
    if logits is not None:
        preds = logits if preds is None else torch.cat((preds, logits), dim=0)
    if labels is not None:
        label_ids = labels if label_ids is None else torch.cat((label_ids, labels), dim=0)


In [36]:
if preds is not None:
    preds = preds.cpu().numpy()
if label_ids is not None:
    label_ids = label_ids.cpu().numpy()

In [37]:
trainer.compute_metrics(EvalPrediction(predictions=preds, label_ids=label_ids))

{'mcc': -0.023180772569263212}

In [41]:
p = EvalPrediction(predictions=preds, label_ids=label_ids)
preds1 = np.argmax(p.predictions, axis=2)
T5_glue_compute_metrics(data_args.task_name, preds1, p.label_ids)

{'mcc': -0.023180772569263212}

In [42]:
label_ids

array([[209],
       [209],
       [209],
       ...,
       [  3],
       [209],
       [209]])

In [48]:
T5_glue_compute_metrics(data_args.task_name, preds1.squeeze()==3, p.label_ids.squeeze()==3)

{'mcc': -0.023303462047471274}

In [46]:
preds1.squeeze()==3

array([ True,  True,  True, ..., False,  True, False])

In [47]:
p.label_ids.squeeze()==3

array([False, False, False, ...,  True, False, False])

In [21]:
sum(label_ids == preds)/len(preds)

array([0.625])

In [22]:
len(preds)

96

In [30]:
l = label_ids.squeeze()

In [33]:
p = preds.squeeze()

In [34]:
sum(l == p)/len(l)

0.625

In [35]:
l

array([209, 209, 209,   3,   3,   3, 209, 209, 209, 209, 209, 209,   3,
       209, 209,   3, 209, 209, 209, 209, 209, 209,   3, 209, 209, 209,
       209,   3,   3, 209, 209,   3,   3, 209, 209, 209, 209, 209,   3,
       209,   3, 209, 209,   3,   3, 209,   3, 209, 209, 209,   3, 209,
       209,   3, 209, 209,   3,   3, 209,   3, 209, 209, 209, 209,   3,
       209, 209,   3, 209, 209, 209, 209,   3, 209,   3,   3,   3,   3,
       209,   3, 209,   3,   3, 209, 209,   3, 209, 209,   3, 209,   3,
       209, 209,   3,   3, 209])

In [36]:
p

array([209, 209, 209,   3, 209,   3, 209, 209,   3, 209, 209, 209, 209,
       209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209,   3,
       209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209,   3,
       209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209,
       209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209,   3,   3,
       209, 209, 209, 209,   3, 209, 209, 209,   3,   3, 209, 209, 209,
       209, 209,   3, 209,   3, 209, 209, 209, 209, 209, 209, 209, 209,
       209, 209, 209, 209,   3])

In [21]:
from sklearn.metrics import matthews_corrcoef, f1_score


In [39]:
matthews_corrcoef(l, p)

0.07972154528942839

In [45]:
l1 = list(map(lambda x: int(x==209), l))

In [46]:
p1 = list(map(lambda x: int(x==209), p))

In [50]:
matthews_corrcoef(l1, p1)

0.07972154528942839

In [51]:
matthews_corrcoef(label_ids, preds)

0.07972154528942839